In [16]:
from rap_db import *
import re
from os import listdir
from os.path import isfile, join
import nltk
#nltk.download()
estconn = pg2.connect(database='rap_songs', user='keenan', host='localhost', password='keenan')
#create_music_tables(estconn)

create_music_tables(estconn, bypass = True)
bulk_load(estconn, ['mf_doom', 'chance_the_rapper'])

In [17]:
doom = basic_lyrc_pull(estconn, 'MF Doom')
chance = basic_lyrc_pull(estconn, 'Chance The Rapper')
em = basic_lyrc_pull(estconn, 'Eminem')

In [18]:
#
#estconn.close()

In [4]:
doom.keys()

dict_keys(['Absolutely', 'Appetizers: One Beer', 'Ballskin', 'Batty-Boys', 'Beef Rapp', 'Blunt Drunk (All Outta Ale Remix)', 'Dead Bent', 'Deep Fried Frenz', 'Doomsday', 'Entrees: Kon Karne', 'Entrees: Kookies', 'Entrees: Vomitspit', 'Gas Drawls', 'Go With the Flow', 'Greenbacks', 'Hey!', 'Hoe Cakes', 'Impostas', 'Is He Ill?', 'Kon Queso', 'Lightworks', 'Meat Grinder', 'Microwave Mayo', "More Rhymin'", 'Rap Ambush', 'Red &amp; Gold', "That's That", 'The Finest'])

"\n* send corrections to the typist\n\n[MF Doom]\nBeef rap\nCould lead to gettin teeth capped\nOr even a wreath for mom dukes on some grief crap\nI suggest ya change ya diet\nIt can lead ta high blood pressure if ya fry it\nOr even a stroke, heart attack, heart disease\nIt ain't no startin back once arteries start ta squeeze\nTake the easy way out phony, until then\nThey know they wouldn't be talkin that bologna in the bullpen\nSo disgustin, pardon self as I discuss this\nThey talk a wealth of shit and they ain't never seen the justice\nBust this, like a cold milk from out the toilet\nTwo batteries some Brillo and some foil, he'a boil it\nHe be better off on PC glued\nAnd it's a feud so don't be in no TV mood\nEvery week it's mystery meat, seaweed stewed [food, we need food!]\n\nHe wears a mask just to cover the raw flesh\nA rather ugly brother with flows that's gorgeous\nDrop dead joints hit the whips like bird shit\nThey need it like a hole in they head or a third tit\nHer bra smell,

In [15]:


#can't just blindly tokenize, you end up hurting info
for k, v in em.items():
    print(v)


[Intro]
Too late for the other side
Caught in a chase 25 to life
Too late for the other side
Caught in a chase 25 to life
Yeah
Too late
I cant keep chasing em
Taking my life like that
Caught in a chase
25 to life

[Verse 1]
I don't think she understands the sacrifices that I've made
Maybe if this bitch had acted right I would've stayed
But I've already wasted over half of my life I would've laid
Down and died for you I no longer cry for you
No more pain bitch you
Took me for granted took my heart and ran it straight into the planet
Into the dirt I can no longer stand it
Now my respect I demand it
I'ma take control of this relationship
Command it, and I'ma be the boss of you now goddamnit
And what I mean is that I will no longer let you control me
So you better hear me out this much you owe me
I gave up my life for you, totally devoted to you, why I've stayed
Faithful all the way this is how I fucking get repaid?
Look at how I dress fucking baggy sweats, go to work a mess
Always in a r

#can't just blindly tokenize, you end up hurting info
for k, v in doom.items():
    tokens = nltk.word_tokenize(v)
    print(k, v, '\n', tokens)
    break

In [ ]:
#need to find the best way to mark of important parts of the song,
# {**} == not sure but one thing is can mean someone saying something
# {?} == dont know line
# [Artist] == i think this is when an artist s speaking
# [yes] == can also mean like () whe someone says somthing, have to use [Capital First letter] to differentiate
#or a great way to differentiate is to check contents of [] against db of artists with tolower and edge case for chorus or outro or intro
# * == annotator notes
# (action!) == i think this is sound of action, can also be someone saying something in track
# () == someone saying something or repeat signal
# (**) ==
# looks like () and [] and {} are pretty much interchanchable and it matters a lot more whats inside
# [???] == i think it means they didn't know
# "" == i think this is when someone else is talking over the track
# Man1: denoting that someone is speaking in a conversation
words = re.sub("([\(\[]).*?([\)\]])", "", lyrics)
words = re.sub("\n","--",words)


"\n* send corrections to the typist\n\n[MF Doom]\nBeef rap\nCould lead to gettin teeth capped\nOr even a wreath for mom dukes on some grief crap\nI suggest ya change ya diet\nIt can lead ta high blood pressure if ya fry it\nOr even a stroke, heart attack, heart disease\nIt ain't no startin back once arteries start ta squeeze\nTake the easy way out phony, until then\nThey know they wouldn't be talkin that bologna in the bullpen\nSo disgustin, pardon self as I discuss this\nThey talk a wealth of shit and they ain't never seen the justice\nBust this, like a cold milk from out the toilet\nTwo batteries some Brillo and some foil, he'a boil it\nHe be better off on PC glued\nAnd it's a feud so don't be in no TV mood\nEvery week it's mystery meat, seaweed stewed [food, we need food!]\n\nHe wears a mask just to cover the raw flesh\nA rather ugly brother with flows that's gorgeous\nDrop dead joints hit the whips like bird shit\nThey need it like a hole in they head or a third tit\nHer bra smell,

In [75]:
class lyrics_text():
    #currently written, some will pick each other up, might be ok, can fix later if needed
    regex_commands = {'[Intro]':'[Intro]',
                      '[Outro]':'[Outro]',
                      '\n':'\n',
                      '[]':'\[(.*?)\]',
                      '{}':'\{(.*?)\}',
                      '**':'\*(.*?)\*',
                      '{**}':'\{\*(.*?)\*\}',
                      '()':'\((.*?)\)',
                      '""':'"(.*?)"',
                      '?':'\?+',
                      '* text \n':'\*[A-Za-z0-9]*(\r\n|\r|\n)'}
    
    def __init__(self, raw_text, artist_name):
        self.raw_text = raw_text
        self.artist_name = artist_name
        self.extras = dict(zip(lyrics_text.regex_commands.keys(), [[]]*len(lyrics_text.regex_commands)))
    
    
    def assign_extras(self):
        #basically want to go through dic and use a re ex to pull out the matching phrase and mark the spot in the string
        #with the marks, I can cut up the lyrics accordingly
        for ext, regex_command in lyrics_text.regex_commands.items():
            #regex stuff, may take time, could be cool to figure out way to build into or json file
            finder = re.compile(regex_command)
            for match in finder:
                #match has a start pos and a match value
                self.extras[ext].append(match.group(), (match.start(), match.start()+len(match.group)))

In [76]:
a = lyrics_text(doom['Beef Rapp'], 'MF Doom')

In [77]:
a.assign_extras()

TypeError: '_sre.SRE_Pattern' object is not iterable

In [73]:
a.extras

{'[Intro]': [],
 '[Outro]': [],
 '\n': [],
 '[]': [],
 '{}': [],
 '**': [],
 '{**}': [],
 '()': [],
 '""': [],
 '?': [],
 '* text \n': []}

In [ ]:
estconn.close()